In [19]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [1]:
class Error(Exception):
    def __init__(self, expr = None, msg = None):
        self.expr = expr
        self.msg = msg
class inputSmallerThanKernel(Error):
    def __init__():
        super(inputSmallerThanKernel, self).__init__()
class nodeDoesNotExist(Error):
    def __init__():
        super(nodeDoesNotExist, self).__init__()

In [7]:
class node(object):
    nodes = []
    def __init__(self, input_shape = (0, 0, 0), output_shape = (0, 0, 0)): # c, i1, i2
        node.nodes.append(self)
        self.no  = len(node.nodes)
        self.in_adj = []
        self.out_adj = []
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.compatible = True
            
    def node_alright(self, curr_node):
        try:
            assert(issubclass(type(curr_node), node))
        except:
            raise Error('Not a node')
# Put this section in graph class
#         try:
#             assert(curr_node in graph_nodes)
#         except:
#             raise nodeDoesNotExist
    
    def determine_compatibility(self):
        for curr_node in self.in_adj:
            curr = (curr_node.output_shape == self.input_shape)
            self.compatible  = self.compatible and curr
            
        for curr_node in self.out_adj:
            curr = (curr_node.input_shape == self.output_shape)
            self.compatible = self.compatible and curr

    def describe_adj_list(self, in_adj, out_adj):
        for curr_node in in_adj + out_adj:
            try:
                node_alright(curr_node)
            except Exception as e:
                raise e
        self.in_adj = in_adj
        self.out_adj = out_adj

    def forward(self, x):
        
            
    def out_shape(self):
        pass
    
    def remove(self):
        ## needed in graph class
        pass
    
    
    def __repr__(self):
        return str(self.no) + " " + str(type(self))

In [8]:
class convolution_block(nn.Module, node):
    all_convs = []
    
    def __init__(self, in_h, in_w, in_channels, out_channels, kernel_size, padding = 0, stride = 1):
        try:
            assert(min(in_h, in_w) +2*padding > kernel_size)
        except:
            raise inputSmallerThanKernel
        super(convolution_block, self).__init__()
        node.__init__(self, (in_channels, in_h, in_w))
        convolution_block.all_convs.append(self)
        self.in_channels  = in_channels
        self.out_channels = out_channels 
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        # NN Layers
        self.conv_layer = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.batch_norm = nn.BatchNorm2d(self.out_channels)
        self.relu = nn.ReLU()
    
    def its_layers(self):
#         conv_layer
#         batch_norm
#         relu_layer
        pass

    def out_shape(self):
        c, h, w = self.input_shape
        C = self.out_channels
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(convolution_block, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 1)
        except:
            raise Error('A convolution block can have only one in-edge')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)

    
    def remove(self): 
        ### Remove from all_convs list
        pass

In [9]:
class max_pool_node(node):
    all_max_pools = []
    
    def __init__(self, in_h, in_w, in_channels, kernel_size, stride = 1, padding = 0):
        try:
            assert(min(in_h, in_w) +2*padding > kernel_size)
        except:
            raise inputSmallerThanKernel
        super(max_pool_node, self).__init__((in_channels, in_h, in_w))
        max_pool_node.all_max_pools.append(self)
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
    
    def out_shape(self):
        c, h, w = self.input_shape
        C = c
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
#     def determine_compatibility(self):
#         super(max_pool_node, self).determine_compatibility()
#         self.compatible  = self.compatible and (len(self.in_adj) == 1)

    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 1)
        except:
            raise Error('A max-pool block can have only one in-edge')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)

    
    def remove(self):
        pass

In [10]:
class merge_node(node):
    all_merge_nodes = []
    
    def __init__(self, parents, child):
        super(merge_node, self).__init__()
        try:
            self.describe_adj_list([parents[0], parents[1]], [child])
        except Exception as e:
            raise e
        merge_node.all_merge_nodes.append(self)
        
    def describe_adj_list(self, in_adj, out_adj):
        try:
            assert(len(in_adj) == 2)
        except:
            raise Error('Parents must be exactly two')
        super(merge_node, self).describe_adj_list(in_adj, out_adj)
        
        
class add_node(merge_node):
    all_add_nodes = []
    
    def __init__(self, parents, child):
        super(add_node, self).__init__(parents, child)
        add_node.all_add_nodes.append(self)
        self.input_shape = self.in_adj[0].output_shape
        self.output_shape = self.out_shape()
    
    def out_shape(self):
        return self.input_shape
    
class concat_node(merge_node):
    # Define Later
    pass

class convex_merge_node(merge_node):
    # Define Later
    pass

In [12]:
class Network(object):
    def __init__(self):
        self.adj_mat = {}
        self.adj_list = {}
        

In [14]:
{a.no:  [b.no], b.no: [a.no]}

{2: [3], 3: [2]}